# BERT Weather Condition Custom Training and Sentiment Analysis Evaluuation

Import the necessary libraries and packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud
import os

Import and read the data

In [2]:
path = "daily_weather_2020.csv" # path to data csv
if(os.path.exists(path)):
    print("File Found")
    df = pd.read_csv(path, usecols=['summary'], low_memory=True)
else:
    print("no file found")

File Found
